In [ ]:
!pip install fasttext

In [ ]:
import fasttext
import re
from nltk.corpus import stopwords
import numpy as np, pandas as pd
from gensim.utils import simple_preprocess

In [ ]:
!pip install nltk
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop = stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


***
Train Set

In [ ]:
df_train = pd.read_csv('train.csv')
df_train.head(2)

,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,Top9,Top10,Top11,Top12,Top13,Top14,Top15,Top16,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25
0,2000-01-03,0,A 'hindrance to operations': extracts from the...,Scorecard,Hughes' instant hit buoys Blues,Jack gets his skates on at ice-cold Alex,Chaos as Maracana builds up for United,Depleted Leicester prevail as Elliott spoils E...,Hungry Spurs sense rich pickings,Gunners so wide of an easy target,Derby raise a glass to Strupar's debut double,"Southgate strikes, Leeds pay the penalty",Hammers hand Robson a youthful lesson,Saints party like it's 1999,Wear wolves have turned into lambs,Stump mike catches testy Gough's taunt,Langer escapes to hit 167,Flintoff injury piles on woe for England,Hunters threaten Jospin with new battle of the...,Kohl's successor drawn into scandal,The difference between men and women,"Sara Denver, nurse turned solicitor",Diana's landmine crusade put Tories in a panic,Yeltsin's resignation caught opposition flat-f...,Russian roulette,Sold out,Recovering a title
1,2000-01-04,0,Scorecard,The best lake scene,Leader: German sleaze inquiry,"Cheerio, boyo",The main recommendations,Has Cubie killed fees?,Has Cubie killed fees?,Has Cubie killed fees?,Hopkins 'furious' at Foster's lack of Hannibal...,Has Cubie killed fees?,A tale of two tails,I say what I like and I like what I say,"Elbows, Eyes and Nipples",Task force to assess risk of asteroid collision,How I found myself at last,On the critical list,The timing of their lives,Dear doctor,Irish court halts IRA man's extradition to Nor...,Burundi peace initiative fades after rebels re...,PE points the way forward to the ECB,Campaigners keep up pressure on Nazi war crime...,Jane Ratcliffe,Yet more things you wouldn't know without the ...,Millennium bug fails to bite


In [ ]:
df_train['Label'] = df_train['Label'].apply(str)

In [ ]:
df_train['tops'] = df_train[df_train.columns[2:]].apply(
    lambda x: ' '.join(x.dropna().astype(str)),
    axis=1
)
df_train = df_train[['tops', 'Label']].rename(columns = {'tops': 'questions', 'Label': 'category'})
df_train.head()

,questions,category
0,A 'hindrance to operations': extracts from the...,0
1,Scorecard The best lake scene Leader: German s...,0
2,Coventry caught on counter by Flo United's riv...,0
3,Pilgrim knows how to progress Thatcher facing ...,1
4,Hitches and Horlocks Beckham off but United su...,1


In [ ]:
# NLP Preprocess
df_train.iloc[:, 0] = df_train.iloc[:, 0].apply(lambda x: ' '.join(simple_preprocess(x)))
df_train.iloc[:, 0] = df_train.iloc[:, 0].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
df_train.head()

,questions,category
0,hindrance operations extracts leaked reports s...,0
1,scorecard best lake scene leader german sleaze...,0
2,coventry caught counter flo united rivals road...,0
3,pilgrim knows progress thatcher facing ban mci...,1
4,hitches horlocks beckham united survive breast...,1


In [ ]:
# Prefixing each row of the category column with '__label__'
df_train.iloc[:, 1] = df_train.iloc[:, 1].apply(lambda x: '__label__' + x)
df_train.head()

,questions,category
0,hindrance operations extracts leaked reports s...,__label__0
1,scorecard best lake scene leader german sleaze...,__label__0
2,coventry caught counter flo united rivals road...,__label__0
3,pilgrim knows progress thatcher facing ban mci...,__label__1
4,hitches horlocks beckham united survive breast...,__label__1


In [ ]:
df_train[['category', 'questions']].to_csv('train.txt', 
                                          index = False, 
                                          sep = ' ',
                                          header = None, 
                                          escapechar = " ")


*** 
Test Set

In [ ]:
df_test = pd.read_csv('test.csv')
df_test.head(2)

,Date,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,Top9,Top10,Top11,Top12,Top13,Top14,Top15,Top16,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25
0,2015-01-02,Most cases of cancer are the result of sheer b...,Iran dismissed United States efforts to fight ...,Poll: One in 8 Germans would join anti-Muslim ...,UK royal family's Prince Andrew named in US la...,Some 40 asylum-seekers refused to leave the bu...,Pakistani boat blows self up after India navy ...,Sweden hit by third mosque arson attack in a week,940 cars set alight during French New Year,Salaries for top CEOs rose twice as fast as av...,"Norway violated equal-pay law, judge says: Jud...",Imam wants radical recruiters of Muslim youth ...,"Saudi Arabia beheaded 83 people in 2014, the m...",'A living hell' for slaves on remote South Kor...,"Worlds 400 richest get richer, adding $92bn in...","Rental Car Stereos Infringe Copyright, Music R...",Ukrainian minister threatens TV channel with c...,Palestinian President Mahmoud Abbas has entere...,Israeli security center publishes names of 50 ...,The year 2014 was the deadliest year yet in Sy...,A Secret underground complex built by the Nazi...,Restrictions on Web Freedom a Major Global Iss...,Austrian journalist Erich Mchel delivered a pr...,Thousands of Ukraine nationalists march in Kiev,Chinas New Years Resolution: No More Harvestin...,Authorities Pull Plug on Russia's Last Politic...
1,2015-01-05,Moscow-&gt;Beijing high speed train will reduc...,Two ancient tombs were discovered in Egypt on ...,China complains to Pyongyang after N Korean so...,Scotland Headed Towards Being Fossil Fuel-Free...,Prime Minister Shinzo Abe said Monday he will ...,Sex slave at centre of Prince Andrew scandal f...,Gay relative of Hamas founder faces deportatio...,The number of female drug addicts in Iran has ...,"After Decades of Searching, the Causeway for t...",India lost 64 tigers in 2014,"Lizard Squad hacking of Sony was ""huge marketi...",Russia 'may face chaos' if extra sanctions imp...,The Saudi air and land forces carry out an att...,ISIL is reportedly setting up a bank in attemp...,Israel to brand anti-assimilation group Lehava...,The Islamic State has approved a 2015 budget o...,"Iceland To Withdraw EU Application, Lift Capit...",Blackfield Capital Founder Goes Missing: The v...,Rocket stage crashes back to Earth in rural Ch...,2 Dead as Aircraft Bombs Greek Tanker in Libya...,Belgian murderer Frank Van Den Bleeken to die ...,Czech President criticizes Ukrainian PM; says ...,3 Vietnamese jets join search for 16 missing F...,France seeks end to Russia sanctions over Ukraine,China scraps rare earths caps


In [ ]:
df_test['questions'] = df_test[df_test.columns[1:]].apply(
    lambda x: ' '.join(x.dropna().astype(str)),
    axis=1
)
df_test = df_test[['questions']]
df_test.head()

,questions
0,Most cases of cancer are the result of sheer b...
1,Moscow-&gt;Beijing high speed train will reduc...
2,US oil falls below $50 a barrel Toyota gives a...
3,'Shots fired' at French magazine HQ 90% of Bib...
4,New Charlie Hebdo issue to come out next week:...


In [ ]:
df_test.iloc[:, 0] = df_test.iloc[:, 0].apply(lambda x: ' '.join(simple_preprocess(x)))
df_test.iloc[:, 0] = df_test.iloc[:, 0].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
df_test.head()

,questions
0,cases cancer result sheer bad luck rather unhe...
1,moscow gt beijing high speed train reduce trip...
2,us oil falls barrel toyota gives away fuel cel...
3,shots fired french magazine hq bibi netanyahus...
4,new charlie hebdo issue come next week hard su...


***
Model

In [ ]:
# Training the fastText classifier
model = fasttext.train_supervised('train.txt')

In [ ]:
# Predicting 
preds = []
for i in range(0,len(df_test)):
  pred = model.predict(df_test.iloc[i,0])
  preds.append(pred[0][0][9])

In [ ]:
out = pd.DataFrame(preds)
out.to_csv('output.csv', index=False)